In [1]:
import pandas as pd
import numpy as np
import faiss
import requests
import gradio as gr
import logging
import os
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer

In [2]:
# --- Setup Logging ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# ────────────── 1. CONFIGURATION & DATA LOADING ──────────────

# UPDATE THIS PATH
csv_path = r"C:\Users\Udayanraje Patil\Downloads\LLM_Model_FInal_data.csv"

In [3]:
try:
    if not os.path.exists(csv_path):
        raise FileNotFoundError(f"File not found: {csv_path}")
    
        # Read CSV with encoding fix
    df = pd.read_csv(
        csv_path, 
        quotechar='"', 
        engine='python',
        encoding='latin1'
    ).fillna("")

    # Fix weird UTF-8/Windows encoding artifacts (only for text columns)
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = (
            df[col]
            .astype(str)
            .str.encode('latin1', errors='ignore')
            .str.decode('utf-8', errors='ignore')
        )

    # 1. Standardize Columns
    df.columns = [c.strip() for c in df.columns]
    
    # 2. Rename columns to standard names
    df.rename(columns={
        'File Name': 'title', 
        'Cause': 'cause', 
        'Direct Link': 'link',
        'Country': 'country',
        'Date': 'date'
    }, inplace=True)

    # 3. Generate Case ID if not present
    if 'case_id' not in df.columns:
        df['case_id'] = df.index + 1

    # 4. Create Embeddings
    model = SentenceTransformer("all-MiniLM-L6-v2")

    search_text = df['title'].astype(str) + " " + df['cause'].astype(str)
    embeds = model.encode(search_text.tolist(), show_progress_bar=True, convert_to_numpy=True).astype("float32")
    
    # 5. Create FAISS Index
    index = faiss.IndexFlatL2(embeds.shape[1])
    index.add(embeds)
    
    logging.info("System Loaded Successfully!")

except Exception as e:
    logging.critical(f"Setup Error: {e}")
    df = pd.DataFrame()
    index = None

2025-12-08 19:56:36,900 - INFO - Use pytorch device_name: cpu
2025-12-08 19:56:36,901 - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2


Batches:   0%|          | 0/20 [00:00<?, ?it/s]

2025-12-08 19:56:48,763 - INFO - System Loaded Successfully!


In [4]:
import numpy as np
import logging

# ────────────────────────── RISK MATRIX ──────────────────────────

LOPA_RISK_MATRIX = {
    1: {1: "LOW", 2: "LOW", 3: "LOW", 4: "MEDIUM", 5: "MEDIUM"},
    2: {1: "LOW", 2: "LOW", 3: "MEDIUM", 4: "MEDIUM", 5: "SERIOUS"},
    3: {1: "LOW", 2: "MEDIUM", 3: "MEDIUM", 4: "SERIOUS", 5: "HIGH"},
    4: {1: "MEDIUM", 2: "MEDIUM", 3: "SERIOUS", 4: "HIGH", 5: "HIGH"},
    5: {1: "MEDIUM", 2: "SERIOUS", 3: "HIGH", 4: "HIGH", 5: "CATASTROPHIC"},
}

CONSEQ_KEYWORDS = {
    "fatal": 5, "death": 5, "explosion": 5, "rupture": 5, "catastrophic": 5,
    "fire": 4, "blast": 4, "release": 4, "vapor": 4, "cloud": 4,
    "leak": 3, "spill": 3, "injury": 3, "burn": 3, "damage": 3, "shutdown": 3,
    "failure": 2, "deviation": 2, "trip": 2, "stuck": 2, "corrosion": 2,
    "alarm": 1, "alert": 1, "warning": 1, "maintenance": 1, "noise": 1
}

def get_consequence_score(text):
    text = str(text).lower()
    scores = [score for kw, score in CONSEQ_KEYWORDS.items() if kw in text]
    return max(scores) if scores else 1


# ────────────────────────── SIMILARITY-BASED FREQUENCY ──────────────────────────

def calculate_frequency_from_similarity(distances):
    """
    similarity = 1 / (1 + distance)
    total_similarity ∈ [0,3]
    frequency rated by interval thresholds.
    """
    d = np.array(distances, dtype=float)
    sim = 1.0 / (1.0 + d)
    total_sim = float(sim.sum())

    if total_sim >= 2.2:
        freq = 5
    elif total_sim >= 1.6:
        freq = 4
    elif total_sim >= 1.4:
        freq = 3
    elif total_sim >= 0.7:
        freq = 2
    else:
        freq = 1

    logging.debug(f"Distances={distances}, Similarities={sim}, total_sim={total_sim:.3f}, freq={freq}")

    return total_sim, freq


# ────────────────────────── FINAL RISK COMPUTATION ──────────────────────────

def calculate_risk(query, similar_df, distances):
    """
    Pure similarity-based model:
    Similarity sum (0–3) used to derive frequency (1–5).
    Consequence from keywords.
    """
    try:
        total_sim, freq = calculate_frequency_from_similarity(distances)
        cons = get_consequence_score(query)
        cat = LOPA_RISK_MATRIX[cons][freq]

        logging.info(f"SimilaritySum={total_sim:.3f}, cons_score={cons}, freq={freq}, risk={cat}")

        return cat, cons, freq

    except Exception as e:
        logging.error(f"Risk Calculation Error: {e}")
        return "LOW (Default)", 1, 1


import matplotlib.pyplot as plt
import numpy as np

def generate_risk_plot(cons_score, freq_score):
    """
    Draws the 5x5 risk matrix and marks the current (Frequency, Consequence) point.
    cons_score: 1–5 (y-axis)
    freq_score: 1–5 (x-axis)
    """
    fig, ax = plt.subplots(figsize=(5, 4))

    # Colors grid: 5x5 (just a simple escalation from green to red)
    colors = np.array([
        [0, 0, 0, 1, 1],
        [0, 0, 1, 1, 2],
        [0, 1, 1, 2, 3],
        [1, 1, 2, 3, 3],
        [1, 2, 3, 3, 4]
    ])
    cmap = plt.cm.colors.ListedColormap(
        ['#66ff66', '#ccff66', '#ffff66', '#ffcc00', '#ff3333']
    )

    # Draw heatmap
    ax.imshow(colors, cmap=cmap, origin="lower", extent=[0.5, 5.5, 0.5, 5.5])

    # Grid lines
    ax.set_xticks(np.arange(1.5, 5.5, 1))
    ax.set_yticks(np.arange(1.5, 5.5, 1))
    ax.grid(color="black", linestyle="-", linewidth=0.5)

    # Mark current point
    ax.scatter(
        freq_score, cons_score,
        color="black", s=200, marker="X",
        edgecolors="white", linewidth=1.5, label="Current Risk"
    )

    ax.set_xlabel("Frequency Rating (1–5)")
    ax.set_ylabel("Consequence Rating (1–5)")
    ax.set_title(f"Risk Position (Freq={freq_score}, Cons={cons_score})")

    ax.set_xlim(0.5, 5.5)
    ax.set_ylim(0.5, 5.5)

    return fig


def llm_call(prompt):
    try:
        # Deterministic generation (temperature=0)
        payload = {
            "model": "tinyllama",
            "prompt": prompt,
            "stream": False,
            "options": {"temperature": 0.0},
        }
        r = requests.post("http://localhost:11434/api/generate",
                          json=payload, timeout=30)
        if r.status_code == 200:
            return r.json().get("response", "").strip()
    except:
        return "AI Analysis Unavailable."

In [5]:
# ────────────── 3. CHATBOT ENGINE ──────────────

def process_query(query):
    # Return structure:
    # [risk_cat, cons, freq, plot, advice, c1_header, c1_cause, c2_header, c2_cause, c3_header, c3_cause]
    empty_result = ["N/A", "0/5", "0/5", None, "", "", "", "", "", "", ""]
    
    if df.empty or index is None:
        return ["System Error: Data not loaded"] + empty_result[1:]
    
    # 1. Search
    q_vec = model.encode([query]).astype("float32")
    distances, indices = index.search(q_vec, k=3)
    valid_idx = [i for i in indices[0] if i < len(df)]
    top_cases = df.iloc[valid_idx]
    
    # 2. Risk Calculation
    risk_cat, cons_score, freq_score = calculate_risk(query, top_cases, distances[0])
    
    # 3. Generate Analysis (Using LLM)
    context = ""
    case_ids = []
    for i, row in top_cases.iterrows():
        context += f"Case: {row.get('title')}\nCause: {row.get('cause')}\n\n"
        case_ids.append(f"#{row.get('case_id')}")
    
    ref_string = ", ".join(case_ids)
    
    prompt = f"""You are a safety expert. Situation: "{query}". 
    Review these similar past accidents: {context}. 
    List 5 critical, immediate safety precautions the operator must take."""
    
    advice_content = llm_call(prompt)
    
    # Final Advice with Case References
    final_advice = f"**References: Analysis based on similar Cases {ref_string}**\n\n{advice_content}"
    
    # 4. Generate Plot
    plot = generate_risk_plot(cons_score, freq_score)
    
    # 5. Format Top 3 Cases (Detailed with Cause Box)
    case_outputs = []
    for i in range(3):
        if i < len(top_cases):
            row = top_cases.iloc[i]
            # Header Info
            cid = row.get('case_id', 'N/A')
            title = row.get('title', 'Unknown Case')
            
            # Country Check
            country = str(row.get('country', 'Not Given')).strip()
            if not country or country.lower() == 'nan': country = "Not Given"
            
            # Date Check
            date = str(row.get('date', 'Not Given')).strip()
            if not date or 'struct' in date.lower() or len(date) < 4: date = "Not Found"
            
            link = row.get('link', '#')
            
            # Markdown Header: Title + Case ID + Details
            header = f"**{title} (Case #{cid})**\n🌍 {country} | 📅 {date} | [🔗 Open Report]({link})"
            
            # Cause Text
            raw_cause = str(row.get('cause', '')).strip()
            if len(raw_cause) < 5:  
                cause_text = "No specific cause detailed in report."
            else:
                cause_text = raw_cause
        else:
            header = "**No Case Found**"
            cause_text = ""
        
        case_outputs.append(header)
        case_outputs.append(cause_text)

    # Return everything
    return [
        risk_cat, 
        f"{cons_score}/5", 
        f"{freq_score}/5", 
        plot, 
        final_advice
    ] + case_outputs


In [6]:
# ────────────── 4. GRADIO BLOCKS UI ──────────────

# CSS to Force Black Text in Cause Boxes
custom_css = """
.cause-box textarea {
    color: black !important;
    background-color: #ffffff !important;
    -webkit-text-fill-color: black !important;
    font-weight: 500;
    border: 1px solid #ccc;
    opacity: 1 !important;
}
.cause-box label {
    font-weight: bold;
    color: #333;
}
"""

with gr.Blocks(title="Safety Risk Advisor", css=custom_css) as demo:
    gr.Markdown("# 🚨 Safety Risk Advisor")
    gr.Markdown("Analyze current process risks using historical accident data.")
    
    with gr.Row():
        # --- LEFT COLUMN: Input & Analysis ---
        with gr.Column(scale=1):
            query_input = gr.Textbox(
                label="Describe Current Situation", 
                placeholder="e.g. High pressure alarm in reactor...",
                lines=3
            )
            analyze_btn = gr.Button("Analyze Risk", variant="primary")
            
            with gr.Group():
                gr.Markdown("### 📊 Risk Assessment")
                with gr.Row():
                    r_cat = gr.Textbox(label="Risk Category")
                    r_cons = gr.Textbox(label="Consequence")
                    r_freq = gr.Textbox(label="Frequency")
            
            gr.Markdown("### 🛡️ AI Recommended Precautions")
            advice_output = gr.Markdown()

        # --- RIGHT COLUMN: Visuals & History ---
        with gr.Column(scale=1):
            gr.Markdown("### 📈 Risk Matrix Visualization")
            risk_plot = gr.Plot(label="Risk Matrix")
            
            gr.Markdown("### 📚 Top 3 Similar Past Accidents")
            
            # Case 1
            with gr.Group():
                c1_header = gr.Markdown()
                c1_cause = gr.Textbox(label="Cause of Accident", lines=3, max_lines=4, elem_classes="cause-box", interactive=False)
            
            # Case 2
            with gr.Group():
                c2_header = gr.Markdown()
                c2_cause = gr.Textbox(label="Cause of Accident", lines=3, max_lines=4, elem_classes="cause-box", interactive=False)
                
            # Case 3
            with gr.Group():
                c3_header = gr.Markdown()
                c3_cause = gr.Textbox(label="Cause of Accident", lines=3, max_lines=4, elem_classes="cause-box", interactive=False)

    # EXAMPLES
    gr.Examples(
        examples=[
            ["Fire incident in the Naphtha Splitter Reboiler Furnace."],
            ["Sudden eruption during adsorbent removal activity."],
            ["Fire outbreak caused by flammable vapor release around the heater coil."],
            ["Level gauge malfunction in the storage tank."]
        ],
        inputs=query_input
    )

    # Wiring
    analyze_btn.click(
        process_query,
        inputs=[query_input],
        outputs=[r_cat, r_cons, r_freq, risk_plot, advice_output, 
                 c1_header, c1_cause, c2_header, c2_cause, c3_header, c3_cause]
    )

if __name__ == "__main__":
    demo.launch()

* Running on local URL:  http://127.0.0.1:7860


2025-12-08 19:56:50,063 - INFO - HTTP Request: GET http://127.0.0.1:7860/gradio_api/startup-events "HTTP/1.1 200 OK"
2025-12-08 19:56:50,097 - INFO - HTTP Request: HEAD http://127.0.0.1:7860/ "HTTP/1.1 200 OK"


* To create a public link, set `share=True` in `launch()`.


2025-12-08 19:56:54,392 - INFO - HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-12-08 19:56:58,239 - INFO - SimilaritySum=1.464, cons_score=1, freq=3, risk=LOW
